<a href="https://colab.research.google.com/github/OUCTheoryGroup/colab_demo/blob/master/MobileNetV1_CIFAR10.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## MobileNet V1

MobileNets: Efficient Convolutional Neural Networks for Mobile Vision Applications, 2017

VGG，GoogleNet，ResNet进一步提高CNN的性能。但是到ResNet，网络已经达到152层，模型大小动辄几百300MB+。这种巨大的存储和计算开销，严重限制了CNN在某些低功耗领域的应用。在实际应用中受限于硬件运算能力与存储（比如几乎不可能在手机芯片上跑ResNet-152），**所以必须有一种能在算法层面有效的压缩存储和计算量的方法。而MobileNet/ShuffleNet正为我们打开这扇窗。**

Mobilenet v1是Google于2017年发布的网络架构，旨在充分利用移动设备和嵌入式应用的有限的资源，有效地最大化模型的准确性，以满足有限资源下的各种应用案例。Mobilenet v1核心是把卷积拆分为Depthwise+Pointwise两部分。

Depthwise 处理一个三通道的图像，使用3×3的卷积核，完全在二维平面上进行，卷积核的数量与输入的通道数相同，所以经过运算会生成3个feature map。卷积的参数为： 3 × 3 × 3 = 27，如下所示：

![Depthwise卷积](http://q6dz4bbgt.bkt.clouddn.com/20200308135119.jpg)

Pointwise 不同之处在于卷积核的尺寸为1×1×k，k为输入通道的数量。所以，这里的卷积运算会将上一层的feature map加权融合，有几个filter就有几个feature map，参数数量为：1 × 1 × 3 × 4 = 12，如下图所示：

![替代文字](http://q6dz4bbgt.bkt.clouddn.com/20200308135426.jpg)

因此，可以看出，同样得到4个feature map，使用Depthwise+Pointwise处理，参数数量可以大大降低。

下面是 MobileNet 可分离卷积部分的代码：


In [0]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import numpy as np
import torch.optim as optim

class Block(nn.Module):
    '''Depthwise conv + Pointwise conv'''
    def __init__(self, in_planes, out_planes, stride=1):
        super(Block, self).__init__()
        # Depthwise 卷积，3*3 的卷积核，分为 in_planes，即各层单独进行卷积
        self.conv1 = nn.Conv2d(in_planes, in_planes, kernel_size=3, stride=stride, padding=1, groups=in_planes, bias=False)
        self.bn1 = nn.BatchNorm2d(in_planes)
        # Pointwise 卷积，1*1 的卷积核
        self.conv2 = nn.Conv2d(in_planes, out_planes, kernel_size=1, stride=1, padding=0, bias=False)
        self.bn2 = nn.BatchNorm2d(out_planes)

    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = F.relu(self.bn2(self.conv2(out)))
        return out

## 创建 DataLoader

In [0]:
# 使用GPU训练，可以在菜单 "代码执行工具" -> "更改运行时类型" 里进行设置
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

transform_train = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))])

transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))])

trainset = torchvision.datasets.CIFAR10(root='./data', train=True,  download=True, transform=transform_train)
testset  = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform_test)

trainloader = torch.utils.data.DataLoader(trainset, batch_size=128, shuffle=True, num_workers=2)
testloader = torch.utils.data.DataLoader(testset, batch_size=128, shuffle=False, num_workers=2)

Extracting ./data/cifar-10-python.tar.gz to ./data
Files already downloaded and verified


## 创建 MobileNetV1 网络

32×32×3 ==> 

32×32×32 ==> 32×32×64 ==> 16×16×128 ==> 16×16×128 ==> 

8×8×256 ==> 8×8×256 ==> 4×4×512 ==> 4×4×512 ==>

2×2×1024 ==> 2×2×1024

接下来为均值 pooling ==> 1×1×1024

最后全连接到 10个输出节点

In [0]:
class MobileNetV1(nn.Module):
    # (128,2) means conv planes=128, stride=2
    cfg = [(64,1), (128,2), (128,1), (256,2), (256,1), (512,2), (512,1), 
           (1024,2), (1024,1)]

    def __init__(self, num_classes=10):
        super(MobileNetV1, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(32)
        self.layers = self._make_layers(in_planes=32)
        self.linear = nn.Linear(1024, num_classes)

    def _make_layers(self, in_planes):
        layers = []
        for x in self.cfg:
            out_planes = x[0]
            stride = x[1]
            layers.append(Block(in_planes, out_planes, stride))
            in_planes = out_planes
        return nn.Sequential(*layers)

    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = self.layers(out)
        out = F.avg_pool2d(out, 2)
        out = out.view(out.size(0), -1)
        out = self.linear(out)
        return out

实例化网络

In [0]:
# 网络放到GPU上
net = MobileNetV1().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(net.parameters(), lr=0.001)

## 模型训练

In [0]:
for epoch in range(10):  # 重复多轮训练
    for i, (inputs, labels) in enumerate(trainloader):
        inputs = inputs.to(device)
        labels = labels.to(device)
        # 优化器梯度归零
        optimizer.zero_grad()
        # 正向传播 +　反向传播 + 优化 
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        # 输出统计信息
        if i % 100 == 0:   
            print('Epoch: %d Minibatch: %5d loss: %.3f' %(epoch + 1, i + 1, loss.item()))

print('Finished Training')

Epoch: 1 Minibatch:     1 loss: 2.287
Epoch: 1 Minibatch:   101 loss: 1.708
Epoch: 1 Minibatch:   201 loss: 1.609
Epoch: 1 Minibatch:   301 loss: 1.589
Epoch: 2 Minibatch:     1 loss: 1.358
Epoch: 2 Minibatch:   101 loss: 1.453
Epoch: 2 Minibatch:   201 loss: 1.202
Epoch: 2 Minibatch:   301 loss: 1.334
Epoch: 3 Minibatch:     1 loss: 1.081
Epoch: 3 Minibatch:   101 loss: 1.158
Epoch: 3 Minibatch:   201 loss: 1.089
Epoch: 3 Minibatch:   301 loss: 1.050
Epoch: 4 Minibatch:     1 loss: 1.072
Epoch: 4 Minibatch:   101 loss: 0.967
Epoch: 4 Minibatch:   201 loss: 1.073
Epoch: 4 Minibatch:   301 loss: 0.984
Epoch: 5 Minibatch:     1 loss: 0.772
Epoch: 5 Minibatch:   101 loss: 0.765
Epoch: 5 Minibatch:   201 loss: 0.950
Epoch: 5 Minibatch:   301 loss: 0.931
Epoch: 6 Minibatch:     1 loss: 0.807
Epoch: 6 Minibatch:   101 loss: 0.659
Epoch: 6 Minibatch:   201 loss: 0.802
Epoch: 6 Minibatch:   301 loss: 0.876
Epoch: 7 Minibatch:     1 loss: 0.678
Epoch: 7 Minibatch:   101 loss: 0.787
Epoch: 7 Min

## 模型测试

In [0]:
correct = 0
total = 0

for data in testloader:
    images, labels = data
    images, labels = images.to(device), labels.to(device)
    outputs = net(images)
    _, predicted = torch.max(outputs.data, 1)
    total += labels.size(0)
    correct += (predicted == labels).sum().item()

print('Accuracy of the network on the 10000 test images: %.2f %%' % (
    100 * correct / total))

Accuracy of the network on the 10000 test images: 77.85 %
